# Importing necessary Libraries

In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

# Importing the dataset

In [ ]:
with open('Brown_train.txt','r') as f:
  lines = f.readlines()
f.close()

In [ ]:
total_lines = len(lines)
print(total_lines)

27491


In [ ]:
print(lines[1])

Using/VERB privately-owned/ADJ vehicles/NOUN was/VERB a/DET personal/ADJ hardship/NOUN for/ADP such/ADJ employees/NOUN ,/. and/CONJ the/DET matter/NOUN of/ADP providing/VERB state/NOUN transportation/NOUN was/VERB felt/VERB perfectly/ADV justifiable/ADJ ./. 



Fetching the tags from the dataset:

In [ ]:
nltk_data = []
for i in range(total_lines):
  master_list = []
  new_list = lines[i].strip(" \n").split(" ")
  for item in new_list:
    temp_list = item.split("/") # list
    temp_tuple = (temp_list[0],temp_list[-1])
    master_list.append(temp_tuple)
  nltk_data.append(master_list)

In [ ]:
print(nltk_data[:2])

[[('At', 'ADP'), ('that', 'DET'), ('time', 'NOUN'), ('highway', 'NOUN'), ('engineers', 'NOUN'), ('traveled', 'VERB'), ('rough', 'ADJ'), ('and', 'CONJ'), ('dirty', 'ADJ'), ('roads', 'NOUN'), ('to', 'PRT'), ('accomplish', 'VERB'), ('their', 'DET'), ('duties', 'NOUN'), ('.', '.')], [('Using', 'VERB'), ('privately-owned', 'ADJ'), ('vehicles', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('personal', 'ADJ'), ('hardship', 'NOUN'), ('for', 'ADP'), ('such', 'ADJ'), ('employees', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('the', 'DET'), ('matter', 'NOUN'), ('of', 'ADP'), ('providing', 'VERB'), ('state', 'NOUN'), ('transportation', 'NOUN'), ('was', 'VERB'), ('felt', 'VERB'), ('perfectly', 'ADV'), ('justifiable', 'ADJ'), ('.', '.')]]


In [ ]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('At', 'ADP')
('that', 'DET')
('time', 'NOUN')
('highway', 'NOUN')
('engineers', 'NOUN')
('traveled', 'VERB')
('rough', 'ADJ')
('and', 'CONJ')
('dirty', 'ADJ')
('roads', 'NOUN')
('to', 'PRT')
('accomplish', 'VERB')
('their', 'DET')
('duties', 'NOUN')
('.', '.')
('Using', 'VERB')
('privately-owned', 'ADJ')
('vehicles', 'NOUN')
('was', 'VERB')
('a', 'DET')
('personal', 'ADJ')
('hardship', 'NOUN')
('for', 'ADP')
('such', 'ADJ')
('employees', 'NOUN')
(',', '.')
('and', 'CONJ')
('the', 'DET')
('matter', 'NOUN')
('of', 'ADP')
('providing', 'VERB')
('state', 'NOUN')
('transportation', 'NOUN')
('was', 'VERB')
('felt', 'VERB')
('perfectly', 'ADV')
('justifiable', 'ADJ')
('.', '.')


In [ ]:
# split data into training and validation set in the ratio 80:20
train_set, test_set = train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [ ]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

435013
108136


In [ ]:
#check how many unique tags are present in training data
tags = []
vocab = []
for item in train_tagged_words:
     tags.append(item[-1])
     vocab.append(item[0])

tags = set(tags)
vocab = set(vocab)
print(len(tags))
print(tags)



12
{'.', 'PRT', 'ADJ', 'PRON', 'ADV', 'NOUN', 'CONJ', 'VERB', 'X', 'NUM', 'ADP', 'DET'}


# Emission Probability

In [ ]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    #now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)


    return (count_w_given_tag, count_tag)

#Transition Probability

In [ ]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

# creating t x t transition matrix of tags, t = no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')

for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        a, b = t2_given_t1(t2, t1)
        tags_matrix[i, j] = a/b
    print(tags_matrix)


[[0.1489525  0.04092313 0.035533   0.12911892 0.08179392 0.12285659
  0.08700961 0.09151011 0.0013955  0.0186823  0.10134841 0.14085859]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.   

In [ ]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,.,PRT,ADJ,PRON,ADV,NOUN,CONJ,VERB,X,NUM,ADP,DET
.,0.148952,0.040923,0.035533,0.129119,0.081794,0.122857,0.087010,0.091510,0.001396,0.018682,0.101348,0.140859
PRT,0.096867,0.014156,0.019644,0.008510,0.045650,0.030539,0.015429,0.580802,0.000000,0.004215,0.102275,0.081915
ADJ,0.111960,0.020652,0.056922,0.004726,0.012056,0.632338,0.039147,0.018255,0.000479,0.005446,0.091993,0.006028
PRON,0.117060,0.024608,0.008877,0.006255,0.054827,0.009153,0.011913,0.696104,0.000046,0.000828,0.052160,0.018168
ADV,0.203091,0.030143,0.125768,0.054004,0.091649,0.030188,0.020291,0.225868,0.000090,0.012880,0.140546,0.065483
NOUN,0.288228,0.018146,0.012145,0.019418,0.028791,0.132998,0.060344,0.171089,0.000352,0.009218,0.243299,0.015973
CONJ,0.022321,0.027623,0.103516,0.081961,0.092285,0.215123,0.000209,0.218820,0.000628,0.021763,0.070871,0.144880
VERB,0.088673,0.071044,0.055459,0.066859,0.102952,0.092135,0.015140,0.182810,0.000209,0.007285,0.164834,0.152599
X,0.297131,0.016393,0.006148,0.002049,0.014344,0.067623,0.024590,0.077869,0.446721,0.000000,0.038934,0.008197
NUM,0.288280,0.004814,0.056471,0.007406,0.018885,0.378448,0.040733,0.045917,0.000185,0.019071,0.127199,0.012590


Viterbi algorithm

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
random.seed(1234)      #define a random seed to get same sentences when run multiple times

rndom = [random.randint(1,len(test_set)) for x in range(10)]

test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]


Testing the Accuracy

In [ ]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]

accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  300.7813310623169
Viterbi Algorithm Accuracy:  91.22807017543859
